데이터 불러오기
---

경로 설정

In [ ]:
from google.colab import drive #코랩에서 구글 드라이브와 연동하기
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
data_path='/content/drive/MyDrive/dataset/'
image_file_list=os.listdir(data_path+'images')        # 이미지 파일 디렉토리
mask_file_list=os.listdir(data_path+'masks')          # 정답 파일 디렉토리 

**데이터 분할**  
학습:검증:테스트 = 6:2:2


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_file_list,mask_file_list,test_size=0.2,shuffle=False,random_state=1004)
X_train,X_val,Y_train,Y_val=train_test_split(X_train,Y_train,test_size=0.2,shuffle=False,random_state=1004)

폴더 생성 함수

학습데이터, 검증데이터, 테스트데이터 나누기.

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):                                             # 디렉토리 만들기. OSError(인수에러)발생시 폴더 만들라는 문구 띄우기.
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

createFolder(data_path+'Original_Train')
createFolder(data_path+'Original_Test')
createFolder(data_path+'Original_val')
createFolder(data_path+'Mask_Train')
createFolder(data_path+'Mask_Test')
createFolder(data_path+'Mask_val')

**생성된 각 폴더에 데이터 복사하기**


 images 폴더에 있는걸 Original_Train폴더로 복사  (X_train)

 masks 폴더에 있는걸 Mask_Train폴더로 복사       (Y_train)

 images 폴더에 있는걸 Original_val폴더로 복사    (X_val)

 masks 폴더에 있는걸 Mask_val폴더로 복사         (Y_val)

 images 폴더에 있는걸 Original_Test폴더로 복사   (X_test)
 
 masks 폴더에 있는걸 Mask_Test폴더로 복사        (Y_test)



In [ ]:
import shutil               # 파일 복사, 이동 등 모듈

src=data_path+'images/'                                               
dir=data_path+'Original_Train/'
for i in range(len(X_train)):
    shutil.copy(src+X_train[i],dir+X_train[i])

src=data_path+'masks/'                                                
dir=data_path+'Mask_Train/'
for i in range(len(Y_train)):
    shutil.copy(src+Y_train[i],dir+Y_train[i])

src=data_path+'images/'
dir=data_path+'Original_val/'
for i in range(len(X_val)):
    shutil.copy(src+X_val[i],dir+X_val[i])
    
src=data_path+'masks/'
dir=data_path+'Mask_val/'
for i in range(len(Y_val)):
    shutil.copy(src+Y_val[i],dir+Y_val[i])

src=data_path+'images/'
dir=data_path+'Original_Test/'
for i in range(len(X_test)):
    shutil.copy(src+X_test[i],dir+X_test[i])

src=data_path+'masks/'
dir=data_path+'Mask_Test/'
for i in range(len(Y_test)):
    shutil.copy(src+Y_test[i],dir+Y_test[i])

generator 단위로 만들기

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


data_gen_args = dict(rotation_range=0.2,                               # 랜덤하게 회전할 각도의 범위
                    width_shift_range=0.05,                            # 왼쪽 오른쪽으로 움직이기(원본 이미지에 대한 비율)
                    height_shift_range=0.05,                           # 위아래로 움직이기(원본 이미지에 대한 비율)
                    shear_range=0.05,                                  # shearing transformation 범위(회전하면서 늘어지는거..?)
                    zoom_range=0.05,                                   # 랜덤 확대 범위
                    horizontal_flip=True,                              # 수평 전환 설정(50%확률로 뒤집는다. 뒤집어도 자연스러운 케이스에 사용하면 좋다함)                      
                    fill_mode='nearest',                               # 빈 공간을 채우는 방식
                    rescale=1./255)                                    # https://laheepapa.tistory.com/70    픽셀값을 0 ~ 1 값으로 맞춰주기 위해 rescale = 1. / 255로 설정

mask_gen_args = dict(rotation_range=0.2,                               ## 검증 및 테스트 이미지는 augmentation을 적용하지 않습니다. 모델 성능을 평가할 때에는 이미지 원본을 사용합니다. 라는데??
                    width_shift_range=0.05,                             #https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

image_datagen = ImageDataGenerator(**data_gen_args)                    # ImageDataGenerator : 기존의 데이터에 랜덤한 변화를 주어 데이터의 수를 증가시켜 학습하게 하는 것. 데이터 증폭. https://velog.io/@usrnamejm/Tensorflow-ImageDataGenerator
mask_datagen  = ImageDataGenerator(**mask_gen_args) 

# Provide the same seed and keyword arguments to the fit and flow methods
#image_datagen.fit(images, augment=True, seed=seed)
#mask_datagen.fit(masks, augment=True, seed=seed)

**flow_from_directory**   
디렉토리에의 경로를 전달받아 증강된 데이터의 배치를 생성

ImageDataGenerator 클래스를 이용하여 객체를 생성한 뒤 flow_from_directory() 함수를 호출하여 제네레이터(generator)를 생성

https://tykimos.tistory.com/13

http://keras-ko.kr/api/preprocessing/image/

In [ ]:
data_path='/content/drive/MyDrive/dataset/'

###train                                                                            # flow_from_directory 메서드는 디스크로부터 이미지를 불러오고, rescaling을 적용하고, 요구되는 면적에 맞게 이미지를 resize 합니다.
image_generator = image_datagen.flow_from_directory(                                 #이렇게 폴더명에 맞춰서 이미지 저장을 하는 이유는 flow_from_directory는 이미지를 불러올 때 폴더명에 맞춰 자동으로 labelling을 해주기 때문입니다
    data_path,                                                                    #이미지 경로
    class_mode=None,                                                      # 라벨 반환되지 않음. binary, categorical, sparse
    classes=['Original_Train'],                                                      #학습데이터 폴더.
    seed=1,                                                                                 #랜덤시드
    batch_size=4,                                                                      # 배치 크기, 한번에 처리를 해야할 데이터의 양을 설정
    target_size=(256,256))                                                       # 패치 이미지 크기. 폴더에 있는 원본 이미지 크기가 다르더라도 target_size에 지정된 크기로 자동 조절 된다.  

mask_generator = mask_datagen.flow_from_directory(
    data_path,
    classes=['Mask_Train'],
    class_mode=None,
    seed=1,
    color_mode='rgb',  #one of "grayscale", "rgb", "rgba". Default: "rgb". Whether the images will be converted to have 1 or 3 color channels.
    batch_size=4,
    target_size=(256,256))

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)             #학습데이터랑, 정답데이터 하나로 zip

###validation
val_image_generator = image_datagen.flow_from_directory(
    data_path,
    class_mode=None,
    classes=['Original_val'],
    seed=1,
    batch_size=4,
    target_size=(256,256))

val_mask_generator = mask_datagen.flow_from_directory(
    data_path,
    classes=['Mask_val'],
    class_mode=None,
    seed=1,
    batch_size=4,
    color_mode='rgb',
    target_size=(256,256))

val_generator = zip(val_image_generator, val_mask_generator)

###test
test_image_generator=image_datagen.flow_from_directory(
    data_path,
    class_mode=None,
    classes=['Original_Test'],
    seed=1,
    batch_size=4,
    color_mode='rgb',
    target_size=(256,256))

test_mask_generator=mask_datagen.flow_from_directory(
    data_path,
    class_mode=None,
    classes=['Mask_Test'],
    seed=1,
    batch_size=4,
    color_mode='rgb',
    target_size=(256,256))

test_generator=zip(test_image_generator,test_mask_generator)

Found 448 images belonging to 1 classes.
Found 448 images belonging to 1 classes.
Found 112 images belonging to 1 classes.
Found 112 images belonging to 1 classes.
Found 141 images belonging to 1 classes.
Found 141 images belonging to 1 classes.


colormap

In [ ]:
import pandas as pd
classes = pd.read_csv(data_path+'class_dict.csv', index_col =0)
classes

,r,g,b
name,,,
Animal,64,128,64
Archway,192,0,128
Bicyclist,0,128,192
Bridge,0,128,64
Building,128,0,0
Car,64,0,128
CartLuggagePram,64,0,192
Child,192,128,64
Column_Pole,192,192,128


In [ ]:
n_classes = len(classes)
n_classes

32

UNET 모델 정의
---
* Unet은 2015년에 출시한  Biomedical Image segmentation을 위한 convolutional network로 출시됨  
* End-to-End 로 Segmentation하는 심플하고 효과적인 방법  
* U-Net은 Fully Convolution Network(FCN)를 기반으로 하여 구축하였으며, 적은 데이터를 가지고도 더욱 정확한 Segmentaion을 내기 위해 FCN 구조를 수정


In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

def unet(n_classes, pretrained_weights = None,input_size = (256,256,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv10 = Conv2D(n_classes, (1,1), activation = 'softmax')(conv9)
    conv10 = Conv2D(n_classes, (1,1), padding='same')(conv9)  

    output_layer = conv10
    output_layer = Activation('softmax')(output_layer)

    model = Model(inputs,output_layer)
    
    #케라스의 MeanIOU를 평가지표로 사용
    model.compile(optimizer = Adam(learning_rate = 1e-4), loss = 'categorical_crossentropy', metrics = [tf.keras.metrics.MeanIoU(num_classes=32)])

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [ ]:
model = unet(n_classes)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


이미지를 숫자형으로 변경
---

In [ ]:
cls2rgb = {cl:list(classes.loc[cl, :]) for cl in classes.index}
cls2rgb

{'Animal': [64, 128, 64],
 'Archway': [192, 0, 128],
 'Bicyclist': [0, 128, 192],
 'Bridge': [0, 128, 64],
 'Building': [128, 0, 0],
 'Car': [64, 0, 128],
 'CartLuggagePram': [64, 0, 192],
 'Child': [192, 128, 64],
 'Column_Pole': [192, 192, 128],
 'Fence': [64, 64, 128],
 'LaneMkgsDriv': [128, 0, 192],
 'LaneMkgsNonDriv': [192, 0, 64],
 'Misc_Text': [128, 128, 64],
 'MotorcycleScooter': [192, 0, 192],
 'OtherMoving': [128, 64, 64],
 'ParkingBlock': [64, 192, 128],
 'Pedestrian': [64, 64, 0],
 'Road': [128, 64, 128],
 'RoadShoulder': [128, 128, 192],
 'SUVPickupTruck': [64, 128, 192],
 'Sidewalk': [0, 0, 192],
 'SignSymbol': [192, 128, 128],
 'Sky': [128, 128, 128],
 'TrafficCone': [0, 0, 64],
 'TrafficLight': [0, 64, 64],
 'Train': [192, 64, 128],
 'Tree': [128, 128, 0],
 'Truck_Bus': [192, 128, 192],
 'Tunnel': [64, 0, 64],
 'VegetationMisc': [192, 192, 0],
 'Void': [0, 0, 0],
 'Wall': [64, 192, 0]}

**adjust_mask(mask, flat)**

이미지를 숫자배열로 변환하는 역할


In [ ]:
import numpy as np

def adjust_mask(mask, flat=False): 
  
    semantic_map = []
    for colour in list(cls2rgb.values()):        
        equality = np.equal(mask, colour) #256x256x3차원을 True/False로 채움
        class_map = np.all(equality, axis = -1) #256x256가 전부 True이면(지정된 픽셀의 색상을 찾으면) True 반환
        semantic_map.append(class_map) #True/False로 256x256 배열의 리스트 생성
    semantic_map = np.stack(semantic_map, axis=-1)# 256x256 배열 x32개 클래스 -> 256x256x32차원으로 만듬
    
    if flat:
        semantic_map = np.reshape(semantic_map, (-1,256*256)) #평면이 아니라면 (-1,256*256)의 2차원으로 변환

    return np.float32(semantic_map)#T/F 배열을 숫자배열로 변환

**generator_fn()**  
generator 내의 모든 값에 adjust_mask함수 적용

In [ ]:
def train_generator_fn():
    for (img,mask) in train_generator:
        new_mask = adjust_mask(mask)
        yield (img,new_mask)

def val_generator_fn():
    for (img,mask) in val_generator:
        new_mask = adjust_mask(mask)
        yield (img,new_mask)

def test_generator_fn():  
    for (img,mask) in test_generator:
        new_mask=adjust_mask(mask)
        yield (img,new_mask)

모델 실행 및 시각화
---

학습, 검증, 평가 과정  진행

In [ ]:
model_checkpoint = ModelCheckpoint('unet_camvid.hdf5', monitor='val_loss',verbose=1, save_best_only=True)

model = unet(n_classes=32)
model.fit_generator(generator=train_generator_fn(),
                    validation_data=val_generator_fn(),
                    validation_steps=100,               
                    steps_per_epoch=1000,
                    epochs=10,
                    callbacks=[model_checkpoint])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
 330/1000 [========>.....................] - ETA: 4:48:46 - loss: 1.8387 - mean_io_u_1: 0.4857

In [ ]:
scores=model.evaluate_generator(test_generator_fn(),steps=5)
scores[1]

시각화

In [ ]:
# img (256,256,3)
# gt_mask: gt_mode=sparse--> (256,256) or ohe --> (256,256,32)
def test_visualize_seg(img, gt_mask, shape='normal'):
    plt.figure(1)

      # Img
    plt.subplot(311)
    plt.imshow(img)

      # Predict
    pred_mask = model.predict(np.expand_dims(img, 0))
    pred_mask = np.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[0]
    if shape=='flat':
        pred_mask = np.reshape(pred_mask, (256,256)) # Reshape only if you use the flat model. O.w. you dont need

    rgb_mask = np.apply_along_axis(map_class_to_rgb, -1, np.expand_dims(pred_mask, -1))

    # Prediction
    plt.subplot(312)
    plt.imshow(rgb_mask)

    plt.subplot(313)
    plt.imshow((gt_mask).astype(np.float64))

In [ ]:
img = next(test_image_generator)[0] #
gt_img = next(test_mask_generator)[0] 
test_visualize_seg(img, gt_img)

영상 만들기

In [ ]:
import re

path = '/content/drive/MyDrive/dataset/Mask_Test'
paths = [os.path.join(path , i ) for i in os.listdir(path) if re.search(".png$", i )]

## 정렬 작업
store1 = []
store2 = []
for i in paths :
    if len(i) == 19 :
        store2.append(i)
    else :
        store1.append(i)

paths = list(np.sort(store1)) + list(np.sort(store2))

In [ ]:
import cv2

#pathIn= '/content/drive/MyDrive/dataset/Mask_Test'
pathOut = '/content/drive/MyDrive/dataset/Mask_Test2.mp4'
fps = 10 #1초당 보여주는 이미지 개수    

frame_array = []
for idx , path in enumerate(paths) : #idx는 반복문 번호
    #if (idx % 2 == 0) | (idx % 5 == 0) :
        #continue
    img = cv2.imread(path) #해당경로에 있는지 이미지 읽기
    height, width, layers = img.shape
    size = (width,height)
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
#cv2.VideoWriter_fourcc(*'DIVX')는 동영상으 코덱, 압축방식, 색상, 픽셀포멧 정의

for i in range(len(frame_array)):
    out.write(frame_array[i]) #이미지 배열에 작성
out.release()